In [ ]:
import numpy as np
import pandas as pd
import random
import datetime
import time

# execution with 1 meter & 1 day: 160 ms
# execution with 1000 meters & 1 day: 3349 ms
# execution with 1 meter & 1000 days: 170 ms
# execution with 1000 meters & 1000 days: 5885 ms
# execution with 10000 meters & 100 days: 40523 ms
# execution with 10000 meters & 1000 days: 809100ms

start_time = time.time()


def create_list_of_meters(number_of_meters):
    random_meters = np.random.randint(10000000, 99999999, number_of_meters).T
    exits_available = ['EA1', 'EA2', 'EA3', 'EA4', 'EM1', 'EM2', 'EM3', 'EM4', 'LC', 'LO', 'LS', 'LT', 'LW', 'NE1',
                       'NE2',
                       'NE3', 'NO1', 'NO2', 'NT1', 'NT2', 'NT3', 'NW1', 'NW2', 'SC1', 'SC2', 'SC4',
                       'SE1', 'SE2', 'SO1', 'SO2',
                       'SW1', 'SW2', 'SW3', 'WA1', 'WA2', 'WM1', 'WM2', 'WM3']
    exits_list = [exits_available[random.randrange(len(exits_available))] for i in range(number_of_meters)]
    aq_random = np.random.randint(0, 2120001, number_of_meters).T
    df = pd.DataFrame({'Meter ID': list(random_meters), 'Estimated Consumption (kWh)': aq_random,
                       'Exit': exits_list}, columns=['Meter ID', 'Estimated Consumption (kWh)', 'Exit'])
    return df


def random_forecast_table(list_of_meters, start_date, duration):
    date_list = [start_date + datetime.timedelta(days=x) for x in range(duration)]
    date_list = np.array(date_list)
    date_list = np.tile(date_list, len(list_of_meters)).T
    meter_list_for_dates = list_of_meters.repeat(duration)
    rnd_consumption = np.random.uniform(low=0.0, high=2000.0, size=(duration * len(list_of_meters),))
    rnd_forecast_table = pd.DataFrame({'Meter_ID': list(meter_list_for_dates), 'Date': list(date_list),
                                       'kwh': rnd_consumption},
                                      columns=['Meter_ID', 'Date', 'kwh'])
    return rnd_forecast_table


# Import data from excel to get the Rate Table
""""""""""""""""
CHANGE THE PATH TO WHERE YOUR EXCEL IS LOCATED
"""""""""""""""""
file = pd.ExcelFile(r'/Users/juliankautsch/PycharmProjects/GorillaAssesment/gorilla_test_data.xlsx')
rate_table = pd.read_excel(file, 'rate_table')
meters_list = []

# Create (X) number of meters with related exit zones and aq
meter_list = create_list_of_meters(1000)
random_meters = meter_list['Meter ID']

# Starting Date for consumption data
YYYY = 2022
MM = 12
DD = 19

# Duration (number of days) for consumption data
duration = 100
forecast_table = random_forecast_table(random_meters, pd.Timestamp(YYYY, MM, DD), duration)
df_final = pd.DataFrame(columns=['Meter ID', 'Estimated Consumption (kWh)', 'Total Cost (£)'])
for i in range(len(meter_list)):
    meter_id = meter_list['Meter ID'].iloc[i]
    aq = meter_list['Estimated Consumption (kWh)'].iloc[i]
    exit_zone = meter_list['Exit'].iloc[i]
    rates_1 = rate_table[(rate_table.exit_zone == exit_zone) & (rate_table.aq_min_kwh <= aq) &
                         ((rate_table.aq_max_kwh >= aq) | (rate_table.aq_max_kwh.isnull()))]
    rate_dates = rates_1['date']
    rate = pd.DataFrame(rates_1['rate_p_per_kwh'])
    forecast_1 = forecast_table[forecast_table.Meter_ID == meter_id]
    estimated_cost = forecast_1['kwh']
    forecast_dates = forecast_1['Date']
    n = 0
    consumption_list = []
    # find starting rate - there aren't too many time periods, so quick solution
    while forecast_dates.iloc[0] > rate_dates.iloc[n]:
        if forecast_dates.iloc[0] < rate_dates.iloc[n + 1]: break
        n += 1
    starting_n = n

    # for starting conditions
    consumption_in_period = 0
    for i in rate_dates:
        last_consumption = consumption_in_period
        if n < len(rate_dates) - 1:
            consumption_in_period = forecast_1.loc[(forecast_1['Date'] >= rate_dates.iloc[n]) &
                                                   (forecast_1['Date'] < rate_dates.iloc[n + 1]), "kwh"].sum()
        else:
            consumption_in_period = forecast_1.loc[(forecast_1['Date'] >= rate_dates.iloc[n]), "kwh"].sum()
        n += 1
        consumption_list.append(consumption_in_period)
        # stop if consumption becomes 0 and there was previous consumption
        # (for start dates not in first rates period)
        # OR when the last RateDate was checked
        if (consumption_in_period == 0) & (last_consumption > 0) | n == len(rate_dates): break
    consumption_df = pd.DataFrame(consumption_list, columns=['kwh'])
    total_cost = round((consumption_df.values * rate.iloc[starting_n:n].values / 100).sum(), 2)
    total_consumption = estimated_cost.sum()
    meter_info = [meter_id, total_consumption, total_cost]
    meters_list.append(meter_info)
df = pd.DataFrame(meters_list, columns=['Meter ID', 'Total Estimated Consumption (kWh)', 'Total Cost (£)'])
print(df)

print("--- %s seconds ---" % (time.time() - start_time))
